In [57]:
from krwordrank.hangle import normalize
from krwordrank.word import KRWordRank
import json

answerDic = {}

def krWordRankFunc(texts):
    wordrank_extractor = KRWordRank(
    min_count = 1, # 단어의 최소 출현 빈도수 (그래프 생성 시)
    max_length = 10, # 단어의 최대 길이
    verbose = True
    )
    
    beta = 0.85    # PageRank의 decaying factor beta
    max_iter = 10
    
    keywords, rank, graph = wordrank_extractor.extract(texts, beta, max_iter)
    return keywords

def findDate(keywords):  #날짜 확인
    dateStr = ""
    ans, temp = 0, 0
    for comp in keywords:
        if comp.find('년') != -1:
            yearnum = comp[0:comp.find('년')]
            if yearnum.isdigit():
                dateStr += yearnum + '-'
                break
    for i in keywords:
        if i.find('월') != -1:
            monthnum = i[0:i.find('월')]
            if monthnum.isdigit():
                dateStr += monthnum.zfill(2) + '-'   
                break
    for i in keywords:
        if i.find('일') != -1:
            daynum = i[0:i.find('일')]
            if daynum.isdigit():
                dateStr += daynum.zfill(2)
                break
    answerDic['날짜'] = dateStr

def findTime(keywords): #시간 확인
    timeStr = ""
    for i in keywords:
        if i.find('시') != -1:
            timenum = i[0:i.find('시')]
            if timenum.isdigit():
                timeStr += timenum.zfill(2) + ':'
                break
            elif timenum == '공공':
                timeStr += '00:'
                break
    for i in keywords:
        if i.find('분') != -1:
            timenum = i[0:i.find('분')]
            if timenum.isdigit():
                timeStr += timenum.zfill(2)
                break
    answerDic['시간'] = timeStr
    
def findInOut(keywords): #입/출항 확인
    for i in keywords:
        if '입항' in i:
            answerDic['입/출항'] = '입항'
            break
        elif '출항' in i:
            answerDic['입/출항'] = '출항'
            break

def findHarborLocation(keywords): #외항/내항 확인
    for i in keywords:
        if '외항' in i:
            answerDic['외/내항'] = '외항'
            break
        elif '내항' in i:
            answerDic['외/내항'] = '내항'
            break

def findShipName(keywords, texts): #선박명 확인
    splitText = texts[0].split(' ')
    breakPoint = False
    for comp in splitText:
        if breakPoint:
            break;
        if '선박명' in comp:
            breakPoint = True;
    for name in keywords:
        if comp in name or name in comp:
            answerDic['선박명'] = name
            break
    
def findShipWeight(keywords, texts): #총톤수 확인
    splitText = texts[0].split(' ')
    splitLen = len(splitText)
    for comp in keywords:
        if comp.find('톤') != -1 and comp[0:comp.find('톤') - 1].isdigit():
            answerDic['총톤수'] = comp
            break
        elif comp.isdigit():
            for i in range(0,splitLen):
                if comp == splitText[i] and '톤' in splitText[i + 1]:
                    answerDic['총톤수'] = comp + '톤'
                    break
            
                    
    

texts = ["선박명 현수호 총톤수는 632톤 2020년 8월 1일 9시 15분에 울산 외항으로 입항할 예정이다"]
texts = [normalize(text, english=False, number=True) for text in texts]
# texts 전처리 영어 미포함, 숫자 미포함 설정

keywords = krWordRankFunc(texts);

# print(keywords)

findInOut(keywords) #입/출항 추출 함수
findHarborLocation(keywords) #외/내항 추출 함수
findDate(keywords) # 날짜 데이터 추출함수
findTime(keywords) # 시간 데이터 추출함수
findShipName(keywords, texts) # 선박명 추출함수
findShipWeight(keywords, texts) # 총톤수 추출함수

with open('./test.json', 'w', encoding='utf-8') as make_file:
    json.dump(answerDic, make_file, ensure_ascii = False)
#json file write

print(texts)
print(answerDic)

scan vocabs ... 
num vocabs = 70
done = 10
['선박명 현수호 총톤수는 632톤 2020년 8월 1일 9시 15분에 울산 외항으로 입항할 예정이다']
{'입/출항': '입항', '외/내항': '외항', '날짜': '2020-08-01', '시간': '09:15', '선박명': '현수호', '총톤수': '632톤'}
